In [ ]:
# Install PySpark and Findspark
!pip install pyspark

In [ ]:
# Install MySQL connector
!wget https://repo1.maven.org/maven2/mysql/mysql-connector-java/8.0.29/mysql-connector-java-8.0.29.jar

In [ ]:
!pip install findspark

In [19]:
import findspark
findspark.init()

from pyspark.sql import SparkSession


In [21]:
# Reconfigure the Spark session to increase maxToStringFields
spark = SparkSession.builder \
    .appName("CSV to MySQL ETL") \
    .config("spark.jars", "mysql-connector-java-8.0.29.jar") \
    .config("spark.sql.debug.maxToStringFields", "100").getOrCreate()

In [23]:
# Replace 'path_to_csv' with the path to your CSV file
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("delimiter", ",") \
    .load("ETL-Pipeline-02/country_wise_latest.csv")

# Show the DataFrame to verify it's loaded correctly
df.head


<bound method DataFrame.head of DataFrame[country_region: string, confirmed: int, deaths: int, recovered: int, active: int, new_cases: int, new_deaths: int, new_recovered: int, deaths_per_100_cases: double, recovered_per_100_cases: double, deaths_per_100_recovered: string, confirmed_last_week: int, one_week_change: int, one_week_percent_increase: double, who_region: string]>

In [27]:
# MySQL database connection details
url = "jdbc:mysql://127.0.0.1:3306/etl_project"  # Corrected the IP address format
properties = {
    "user": "anwar",
    "password": "qrc135zx",
    "driver": "com.mysql.cj.jdbc.Driver"
}

# Write the DataFrame to the MySQL table
df.write.jdbc(url=url, table="country_wise", mode="append", properties=properties)


In [29]:
spark.stop()